In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-purchase-history-from-electronics-store/kz.csv


In [2]:
import sqlite3
import csv

data = pd.read_csv('/kaggle/input/ecommerce-purchase-history-from-electronics-store/kz.csv')

data.head(5)

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18


In [3]:
conn = sqlite3.connect("test.db")
data.to_sql('kz', conn)

In [4]:
#example of SQL script 
cur = conn.cursor()
cur.execute("SELECT * FROM kz")
print(cur.fetchone())

(0, '2020-04-24 11:50:39 UTC', 2294359932054536986, 1515966223509089906, 2.268105426648171e+18, 'electronics.tablet', 'samsung', 162.01, 1.515915625441994e+18)


In [5]:
df = pd.read_sql("SELECT order_id FROM kz", conn, index_col=None)
df.head(5)

,order_id
0,2294359932054536986
1,2294359932054536986
2,2294444024058086220
3,2294444024058086220
4,2294584263154074236


In [6]:
#Calculating the feature quantity for the entire table
'''
df = pd.read_sql("SELECT 
				   count(*) AS total_count
				  , count(distinct product_id) as product_count 
				  , count(distinct order_id) as order_count
				  , count(distinct category_id) as category_count
				  , count(distinct user_id) as user_count
				  , sum(price) as sum
				  , AVG(price) as avg
				  , max(price) as max
				  , min(price) as min
				 from kz")
				 
'''

df = pd.read_sql("SELECT  count(*) AS total_count, count(distinct product_id) as product_count , count(distinct order_id) as order_count, count(distinct category_id) as category_count, count(distinct user_id) as user_count, sum(price) as sum, AVG(price) as avg, max(price) as max, min(price) as min from kz", conn, index_col=None)

df

,total_count,product_count,order_count,category_count,user_count,sum,avg,max,min
0,2633521,25113,1435266,900,98262,3.392464e+08,154.093165,50925.9,0.0


In [7]:
#Calculating the feature quantity for users
'''
df = pd.read_sql("SELECT  user_id,
					count(*) AS total_count
				  , count(distinct product_id) as product_count 
				  , count(distinct order_id) as order_count
				  , count(distinct category_id) as category_count				  
				  , sum(price) as sum
				  , AVG(price) as avg
				  , max(price) as max
				  , min(price) as min
				 from kz
				 GROUP BY user_id 
				 order by sum desc
")
				 

'''
df = pd.read_sql("SELECT  user_id, count(*) AS total_count, count(distinct product_id) as product_count, count(distinct order_id) as order_count, count(distinct category_id) as category_count, sum(price) as sum, AVG(price) as avg, max(price) as max, min(price) as min from kz GROUP BY user_id order by sum desc", conn, index_col=None)
df

,user_id,total_count,product_count,order_count,category_count,sum,avg,max,min
0,NaN,2069352,17917,1056723,657,2.217473e+08,135.426653,50925.90,0.00
1,1.515916e+18,635,460,402,134,1.217013e+05,191.655591,1157.38,0.23
2,1.515916e+18,632,521,364,154,1.118839e+05,177.031472,1736.09,0.00
3,1.515916e+18,584,493,328,140,1.088258e+05,186.345599,4745.35,0.67
4,1.515916e+18,548,470,342,132,9.496708e+04,173.297591,3009.24,0.23
...,...,...,...,...,...,...,...,...,...
98258,1.515916e+18,1,1,1,1,0.000000e+00,0.000000,0.00,0.00
98259,1.515916e+18,1,1,1,1,0.000000e+00,0.000000,0.00,0.00
98260,1.515916e+18,1,1,1,1,0.000000e+00,0.000000,0.00,0.00
98261,1.515916e+18,1,1,1,1,0.000000e+00,0.000000,0.00,0.00
